In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
import pickle
from PIL import Image
from tensorboardX import SummaryWriter

In [2]:
# for CRNN
class Dataset_CRNN(data.Dataset):
    "Characterizes a dataset for PyTorch"
    def __init__(self, data_path, frame_length=10, transform=None):
        "Initialization"
        self.transform = transform
        #self.frames = frames
        self.folders = data_path
        self.frames = frame_length #For our case since we are computing 10 frames always

    def __len__(self):
        "Denotes the total number of samples"
        return len(os.listdir(self.folders))

    def read_images(self, data_path, use_transform):
        X = []
        file_name = ""
        for i in os.listdir(data_path):
            file_name = i
            image = Image.open(os.path.join(data_path,i))
            
            #print(image.shape)
            if use_transform is not None:
                image = use_transform(image)
                #print(image.size)
            image = torch.from_numpy(np.asarray(image))
            X.append(image)
        X = torch.stack(X, dim=0)

        return X, file_name

    def __getitem__(self, index):
        data_path = os.path.join(self.folders,os.listdir(self.folders)[index])
              
        # Load data
        X, file_name = self.read_images(data_path, self.transform)                     # (input) spatial images
        
        y = np.ones(self.frames)
        if 'orig' in data_path or 'real' in data_path:
            y = np.zeros(self.frames)
        #print("Folder is {}".format(data_path))
        #print(X.shape)
        return X, torch.from_numpy(y).type(torch.LongTensor), file_name

In [3]:
TRANSFORM_IMG_TRAIN = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.RandomResizedCrop(256)
    
    ])

In [4]:
TRANSFORM_IMG_VAL = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256)
])

In [5]:
train_path = '/home/chinmay/datatset/combined_dataset/train/'
train_data = Dataset_CRNN(train_path, transform=TRANSFORM_IMG_TRAIN, frame_length=10 )
# for step, (x, y) in enumerate(data):
#     print(x.shape)
val_path = '/home/chinmay/datatset/combined_dataset/val'
val_data = Dataset_CRNN(val_path, transform=TRANSFORM_IMG_VAL, frame_length=10)

In [6]:
batch_size = 32
epochs = 40
log_interval = 20
learning_rate = 1e-4

In [7]:
from Meso4 import Meso4

In [8]:
# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu") 
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = data.DataLoader(train_data, **params)
valid_loader = data.DataLoader(val_data, **params)
#model = Meso4()
#model.cuda()
# for index, (X,y) in enumerate(train_loader):
#     y_pred = model(X)
#     print (y_pred.shape)
# Shape was torch.Size([4, 10, 2])
model = Meso4()
model.cuda()

Meso4(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (conv2): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool_2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (max_pool_4): MaxPool2d(kernel_size=(4, 4), stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc_conv): Conv2d(16, 2, kernel_size=(16, 16), stride=(1, 1))
  (dropout): Dropout2d(p=0.

In [9]:
PATH = "/home/chinmay/model_weights/meso_combined/meso_df_2_epoch30.pth"
model.load_state_dict(torch.load(PATH))

In [10]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
def find_median(numpy_array = []): #This is expected to take an array of array. So,
    #print("Input array is {}".format(numpy_array))
    output = []
    confidence_scores = []
    for array in numpy_array:
        counts = np.bincount(array)
        output.append(np.argmax(counts))
        # Let us compute the confidence of the scores
        # since frames are independent, our confidence is purely based on the number
        # of frames our model thinks is belonging to a specific category
        # the confidence of individual frame prediction is not taken into consideration
        # and this portion is debatable....
        frame_set_pred = np.sort(counts)[-1]
        confidence = frame_set_pred/sum(counts)
        confidence_scores.append(confidence)
    return torch.from_numpy(np.asarray(output)).type(torch.LongTensor), torch.from_numpy(np.asarray(confidence_scores)).type(torch.FloatTensor)

In [12]:
writer_train = SummaryWriter('/home/chinmay/training-results/mesonet_mit_scores_combined/train')
writer_test = SummaryWriter('/home/chinmay/training-results/mesonet_mit_scores_combined/test')
save_model_path = "/home/chinmay/model_weights/meso_combined"
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train() #Put the model in training mode
    losses = []
    N_count = 0   # counting total trained sample in one epoch
    scores = []
    #single_iter_loss = []
    for batch_idx, (X, y, file_name) in enumerate(train_loader):
        # distribute data to device
        X, y = X.to(device), y.to(device)
        N_count += X.size(0)

        optimizer.zero_grad()
        output = model(X)   # output has dim = (batch, number of classes)
        #print(output)
        #print(y.shape)
        loss_train = 0
        for items in range(output.shape[1]):
            loss = loss_function(output[:,items,:], y[:,items])
            loss_train = loss_train + loss.item()
            #single_iter_loss.append(loss)
            loss.backward(retain_graph=True)
            optimizer.step()
        #print("backward update")
        #loss_batch = np.sum(single_iter_loss)
        #loss_batch.backward(retain_graph=True) #Possible improvement to increase speed
        #optimizer.step()
        
        losses.append(loss_train)

        # to compute accuracy
        y_pred = torch.max(output, 2)[1]  # y_pred != output score note used here
        # We take the modal value of all predictions. So,
        y, _ = find_median(y) #score param is meaningless
        y_pred, _ = find_median(y_pred)
        #print("filename is {}".format(file_name))
        #print("prediction is {}".format(y_pred))
        
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU
        #print(step_score)
        

        # show information
                 
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss_train, 100 * step_score))
        
    # we should sum all these losses and then divide by the training batch size
    losses_scalar = np.sum(losses)/ len(train_loader.dataset) 
    return losses_scalar, np.mean(scores)

In [13]:
def validation(model, device, optimizer, test_loader):
    # set model as testing mode
    model.eval()

    test_loss = 0
    all_y = []
    all_y_pred = []
    with torch.no_grad():
        for X, y,file_name in test_loader:
            # distribute data to device
            X, y = X.to(device), y.to(device)

            output = model(X)
            #print("Output shape is {}".format(output.shape))
            #print("Expected shape is {}".format(y.shape))
            for items in range(output.shape[1]):
                loss = loss_function(output[:,items,:], y[:,items])
                test_loss += loss.item()                 # sum up batch loss
            #Predict value
            y_pred = torch.max(output, 2)[1]  # (y_pred != output) get the index of the max log-probability
            # Replace the value with the median value
            y_pred, score = find_median(y_pred)
            y, _ = find_median(y) #score param is meaningless
            
            # print few results
            print("Prediction is {} and score is {}".format(y_pred,score))
            
            
            # print("actual value of labels read from disc is {}".format(y))
            # print("filename is {} and prediction is {}".format(file_name, y_pred))
            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss /= len(test_loader.dataset)

    # compute accuracy
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())

    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))

    # save Pytorch models of best record
    torch.save(model.state_dict(), os.path.join(save_model_path, 'meso_df_2_epoch{}.pth'.format(epoch + 1)))  # save spatial_encoder
    torch.save(optimizer.state_dict(), os.path.join(save_model_path, 'meso_df_2_optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
    

    return test_loss, test_score

In [14]:
def adjust_learning_rate(optimizer, learning_rate, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    lr = learning_rate * (0.1 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [15]:
epoch_train_losses = []
epoch_train_scores = []
epoch_test_losses = []
epoch_test_scores = []
for epoch in range(epochs):
    # train, test model
    train_losses, train_scores = train(log_interval, model, device, train_loader, optimizer, epoch)
    epoch_test_loss, epoch_test_score = validation(model, device, optimizer, valid_loader)
    # Reduce learning-rate by a factor of 1/10 after every 10 epochs
    adjust_learning_rate(optimizer=optimizer, learning_rate=learning_rate, epoch=epoch)
    # save results
    writer_train.add_scalar('loss',train_losses,epoch+1)
    writer_train.add_scalar('score',train_scores,epoch+1)
    writer_test.add_scalar('loss',epoch_test_loss,epoch+1)
    writer_test.add_scalar('score',epoch_test_score,epoch+1)
    epoch_train_losses.append(train_losses)
    epoch_train_scores.append(train_scores)
    epoch_test_losses.append(epoch_test_loss)
    epoch_test_scores.append(epoch_test_score)
    #Empty the cache
    torch.cuda.empty_cache()


Train Epoch: 1 [640/2643 (24%)]	Loss: 6.778712, Accu: 46.88%
Train Epoch: 1 [1280/2643 (48%)]	Loss: 6.567217, Accu: 68.75%
Train Epoch: 1 [1920/2643 (72%)]	Loss: 6.183667, Accu: 75.00%
Train Epoch: 1 [2560/2643 (96%)]	Loss: 5.813152, Accu: 78.12%
Prediction is tensor([0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        1, 1, 1, 0, 0, 0, 1, 0]) and score is tensor([1.0000, 0.9000, 0.6000, 0.9000, 1.0000, 0.8000, 0.6000, 0.8000, 0.6000,
        1.0000, 0.7000, 0.9000, 0.5000, 1.0000, 0.8000, 1.0000, 1.0000, 0.9000,
        0.8000, 0.9000, 0.5000, 0.8000, 1.0000, 1.0000, 0.9000, 0.8000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
Prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 1, 1, 0, 1, 1, 0, 1]) and score is tensor([1.0000, 0.7000, 1.0000, 1.0000, 0.8000, 1.0000, 0.9000, 1.0000, 0.9000,
        1.0000, 0.7000, 0.7000, 1.0000, 1.0000, 0.9000, 1.0000, 0.5000, 0.6000,
        0.9000, 1.0000, 

Prediction is tensor([0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0]) and score is tensor([0.8000, 1.0000, 1.0000, 1.0000, 0.6000, 1.0000, 0.8000, 0.9000, 0.7000,
        0.8000, 1.0000, 1.0000, 1.0000, 0.9000, 1.0000, 1.0000, 1.0000, 0.9000,
        0.7000, 0.9000, 0.7000, 1.0000, 1.0000, 0.9000, 0.6000, 1.0000, 1.0000,
        1.0000, 0.8000, 1.0000, 1.0000, 0.5000])
Prediction is tensor([1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0]) and score is tensor([0.8000, 0.6000, 0.7000, 1.0000, 1.0000, 0.5000, 1.0000, 1.0000, 0.9000,
        1.0000, 0.7000, 0.8000, 1.0000, 0.6000, 0.6000, 0.8000, 0.7000, 0.6000,
        0.9000, 1.0000, 0.5000, 0.9000, 1.0000, 0.8000, 0.7000, 0.9000, 1.0000,
        0.8000, 0.8000, 1.0000, 0.8000, 0.9000])
Prediction is tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 1]) and score is te

Prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]) and score is tensor([0.6000, 0.9000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.9000, 1.0000, 1.0000, 1.0000, 0.8000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
Prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0]) and score is tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 0.9000, 0.7000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        0.8000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9000, 0.6000, 1.0000, 0.7000,
        1.0000, 0.6000, 0.9000, 0.5000, 1.0000])
Prediction is tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]) and score is te

Prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0]) and score is tensor([0.9000, 1.0000, 1.0000, 1.0000, 0.7000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 0.8000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.5000, 1.0000, 1.0000])
Prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]) and score is tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.5000, 1.0000,
        0.7000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.8000, 0.9000, 0.9000,
        1.0000, 1.0000, 1.0000, 0.7000, 1.0000, 0.7000, 0.8000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 0.8000])
Prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]) and score is te

Prediction is tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0]) and score is tensor([0.9000, 1.0000, 1.0000, 1.0000, 0.8000, 0.9000, 0.7000, 1.0000, 1.0000,
        1.0000, 0.9000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9000, 1.0000, 0.8000,
        0.5000, 1.0000, 1.0000, 0.9000, 1.0000, 0.7000, 0.9000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.6000, 1.0000, 1.0000])
Prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 0]) and score is tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.6000, 1.0000, 0.7000, 0.9000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9000, 0.8000, 0.7000,
        0.9000, 0.8000, 0.9000, 0.9000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        0.9000, 1.0000, 0.5000, 0.9000, 1.0000])
Prediction is tensor([0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0]) and score is te

...

Prediction is tensor([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 0, 1, 1]) and score is tensor([0.6000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9000, 1.0000, 0.9000, 0.8000,
        0.5000, 1.0000, 0.7000, 1.0000, 1.0000, 1.0000, 1.0000, 0.6000, 1.0000,
        0.6000, 0.6000, 1.0000, 1.0000, 0.7000, 0.7000, 0.9000, 0.9000, 1.0000,
        0.9000, 0.8000, 1.0000, 1.0000, 0.8000])
Prediction is tensor([1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 1]) and score is tensor([0.7000, 0.8000, 0.8000, 0.7000, 1.0000, 0.9000, 1.0000, 1.0000, 0.8000,
        1.0000, 1.0000, 0.8000, 1.0000, 0.8000, 0.8000, 0.6000, 0.9000, 1.0000,
        0.6000, 1.0000, 0.6000, 1.0000, 0.6000, 0.5000, 1.0000, 0.9000, 0.8000,
        1.0000, 0.8000, 0.9000, 0.8000, 1.0000])
Prediction is tensor([0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0]) and score is te

Prediction is tensor([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 1, 1, 1, 0]) and score is tensor([0.8000, 0.7000, 0.6000, 0.7000, 1.0000, 0.5000, 0.9000, 1.0000, 1.0000,
        0.9000, 0.6000, 0.9000, 0.5000, 1.0000, 0.9000, 1.0000, 0.9000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 0.9000, 1.0000, 1.0000, 1.0000, 0.8000,
        0.6000, 1.0000, 0.8000, 0.9000, 0.5000])
Prediction is tensor([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 1, 0, 1]) and score is tensor([1.0000, 0.9000, 0.9000, 1.0000, 0.9000, 0.7000, 1.0000, 0.9000, 0.8000,
        1.0000, 0.8000, 0.9000, 1.0000, 1.0000, 1.0000, 1.0000, 0.7000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9000, 0.8000, 0.9000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
Prediction is tensor([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 0, 1, 1, 1, 0, 1]) and score is te

Process Process-145:
Process Process-148:
Process Process-146:
Process Process-147:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/chinmay/anaconda3/lib/python3.5/

Traceback (most recent call last):
  File "/home/chinmay/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-983838bf8038>", line 7, in <module>
    train_losses, train_scores = train(log_interval, model, device, train_loader, optimizer, epoch)
  File "<ipython-input-12-51bb003cfe2f>", line 10, in train
    for batch_idx, (X, y, file_name) in enumerate(train_loader):
  File "/home/chinmay/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 330, in __next__
    idx, batch = self._get_batch()
  File "/home/chinmay/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 309, in _get_batch
    return self.data_queue.get()
  File "/home/chinmay/anaconda3/lib/python3.5/queue.py", line 164, in get
    self.not_empty.wait()
  File "/home/chinmay/anaconda3/lib/python3.5/threading.py", line 293, in wait
    waiter.acquire()
Keyboa

KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# E